In [5]:
!pip install numpy

In [8]:
import numpy as np

In [9]:
import pandas as pd

In [14]:
!conda install -c anaconda netcdf4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/dprakas1/anaconda3

  added / updated specs:
    - netcdf4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.5.15  |                0         133 KB  anaconda
    certifi-2019.6.16          |           py37_0         154 KB  anaconda
    cftime-1.0.3.4             |py37h1d22016_1001         257 KB  anaconda
    conda-4.7.11               |           py37_0         3.0 MB  anaconda
    hdf4-4.2.13                |       h39711bb_2         874 KB  anaconda
    libnetcdf-4.6.1            |       hd5207e6_2         1.2 MB  anaconda
    netcdf4-1.4.2              |   py37h13743db_0         444 KB  anaconda
    opens

In [15]:
import pandas as pd
import numpy as np
import dask.array as da
import dask.dataframe as dd
import time
import math
#import graphviz
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import h5py



In [16]:
def read_filelist(loc_dir,prefix,unie,fileformat):
    # Read the filelist in the specific directory
    str = os.popen("ls "+ loc_dir + prefix + unie + "*."+fileformat).read()
    fname = np.array(str.split("\n"))
    fname = np.delete(fname,len(fname)-1)
    
    return fname

def read_MODIS(fname1,fname2,verbose=False): # READ THE HDF FILE
    # Read the cloud mask from MYD06_L2 product')
    ncfile=Dataset(fname1,'r')
    CM1km = np.array(ncfile.variables['Cloud_Mask_1km'])
    CM   = (np.array(CM1km[:,:,0],dtype='byte') & 0b00000110) >>1
    #ncfile = Dataset(fname1, "r")
    #CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    #CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    ncfile.close()
    
    ncfile=Dataset(fname2,'r')
    lat  = np.array(ncfile.variables['Latitude'])
    lon  = np.array(ncfile.variables['Longitude'])
    #ncfile = Dataset(MOD03_files, "r")
    #latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    attr_lat = ncfile.variables['Latitude']._FillValue
    attr_lon = ncfile.variables['Longitude']._FillValue
    """#Use _FillValue to remove fill data in lat & lon
    lat[np.where(lat == attr_lat)] = 0.0
    lon[np.where(lat == attr_lat)] = 0.0
    CM [np.where(lat == attr_lat)] = 0.5 #which will not be identified by lines 80-83 
    lat[np.where(lon == attr_lon)] = 0.0
    lon[np.where(lon == attr_lon)] = 0.0
    CM [np.where(lon == attr_lon)] = 0.5 #which will not be identified by lines 80-83
    ncfile.close()"""
    return lat,lon,CM



In [17]:
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')
    
    for i in np.arange(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

def countzero(x, axis=1):
    #print(x)
    count0 = 0
    count1 = 0
    for i in x:
        if i <= 1:
            count0 +=1
    #print(count0/len(x))
    return count0/len(x)

satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
lat_bnd = np.arange(-90,91,1)# latitude and longtitude boundaries of level-3 grid
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)



In [49]:
MYD06_dir= '/Users/dprakas1/Desktop/modis_oneday/'
MYD06_prefix = 'MYD06_L2.A2008'
MYD03_dir= '/Users/dprakas1/Desktop/modis_oneday/'
MYD03_prefix = 'MYD03.A2008'
fileformat = 'hdf'

fname1,fname2 = [],[]


days = np.arange(1,31,dtype=np.int)
for day in days:
    dc ='%03i' % day
    fname_tmp1 = read_filelist(MYD06_dir,MYD06_prefix,dc,fileformat)
    fname_tmp2 = read_filelist(MYD03_dir,MYD03_prefix,dc,fileformat)
    fname1 = np.append(fname1,fname_tmp1)
    fname2 = np.append(fname2,fname_tmp2)

# Initiate the number of day and total cloud fraction
files  = np.arange(len(fname1))



for j in range(0,1):#hdfs:
    print('steps: ',j+1,'/ ',(fname1)) 

    # Read Level-2 MODIS data
    lat,lon,CM = read_MODIS(fname1[j],fname2[j])
print((fname1))
print((fname2))
#rint(CM)
#lat = lat.ravel()
#lon = lon.ravel()
#CM  = CM.ravel()
CM.shape    



steps:  1 /  ['/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0000.006.2013341193524.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0005.006.2013341193207.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0010.006.2013341192125.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0015.006.2013341193150.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0020.006.2013341193235.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0025.006.2013341192801.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0030.006.2013341193002.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0035.006.2013341193634.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0040.006.2013341200201.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0045.006.2013341200648.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0050.006.2013341205630.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A200

['/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0000.006.2013341193524.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0005.006.2013341193207.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0010.006.2013341192125.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0015.006.2013341193150.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0020.006.2013341193235.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0025.006.2013341192801.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0030.006.2013341193002.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0035.006.2013341193634.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0040.006.2013341200201.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0045.006.2013341200648.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0050.006.2013341205630.hdf'
 '/Users/dprakas1/Desktop/modis_oneday/MYD06_L2.A2008001.0055.006

(2030, 1354)

In [50]:
cm = np.zeros((2030,1354), dtype=np.float32)

for MOD06_file in fname1:
    #print(MOD06_file)
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,0]# Reading Specific Variable 'Cloud_Mask_1km'.
    CM = (np.array(CM,dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    #print(CM.shape)
    cm = np.concatenate((cm,CM))
    #bit0 = np.dstack((bit0,bit0r))
    #bit12 = np.dstack((bit12,bit12r))
    
print('The Cloud Mask Array Shape Is: ',cm.shape)


The Cloud Mask Array Shape Is:  (586950, 1354)


In [51]:
lat = np.zeros((2030,1354), dtype=np.float32)
lon = np.zeros((2030,1354), dtype=np.float32)
for MOD03_file in fname2:
    #print(MOD03_file)
    myd03 = Dataset(MOD03_file, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = np.concatenate((lat,latitude))


    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    lon = np.concatenate((lon,longitude))
    
print('Longitude Shape Is: ',lon.shape)
print('Latitude Shape Is: ',lat.shape)



Longitude Shape Is:  (586950, 1354)
Latitude Shape Is:  (586950, 1354)


In [52]:
cm=np.ravel(cm)
lat=np.ravel(lat)
lon=np.ravel(lon)


In [62]:
lon

masked_array(data=[   0.     ,    0.     ,    0.     , ..., -163.16243,
                   -163.22868, -163.29547],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [55]:
d = {'Latitude' :  pd.Series(lat), 'Longitude' : pd.Series(lon),'CM':pd.Series(cm)} #Reading the values in a pandas dataframe
df = pd.DataFrame(d,columns=['Latitude', 'Longitude','CM']) #Creating a pandas dataframe


In [56]:
df

,Latitude,Longitude,CM
0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.0
2,0.000000,0.000000,0.0
3,0.000000,0.000000,0.0
4,0.000000,0.000000,0.0
5,0.000000,0.000000,0.0
6,0.000000,0.000000,0.0
7,0.000000,0.000000,0.0
8,0.000000,0.000000,0.0
9,0.000000,0.000000,0.0


In [57]:
df['Longitude'].unique

<bound method Series.unique of 0              0.000000
1              0.000000
2              0.000000
3              0.000000
4              0.000000
5              0.000000
6              0.000000
7              0.000000
8              0.000000
9              0.000000
10             0.000000
11             0.000000
12             0.000000
13             0.000000
14             0.000000
15             0.000000
16             0.000000
17             0.000000
18             0.000000
19             0.000000
20             0.000000
21             0.000000
22             0.000000
23             0.000000
24             0.000000
25             0.000000
26             0.000000
27             0.000000
28             0.000000
29             0.000000
                ...    
794730270   -161.547974
794730271   -161.602463
794730272   -161.657318
794730273   -161.712555
794730274   -161.768143
794730275   -161.824127
794730276   -161.880478
794730277   -161.937225
794730278   -161.994370
794730279

In [67]:
df=df.astype(np.int)
df

,Latitude,Longitude,CM
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,0,0,0
9,0,0,0


In [68]:
df['Longitude'].unique()

array([   0,   15,   16,   17,   18,   19,   20,   21,   22,   23,   24,
         25,   26,   27,   28,   29,   30,   31,   32,   33,   34,   35,
         36,   37,   38,   39,   40,   41,   42,   43,   44,   45,   46,
         47,   48,   49,   50,   51,   52,   53,   54,   55,   56,   57,
         58,   59,   60,   61,   62,   63,   64,   65,   66,   14,   13,
         12,   11,   10,    9,    8,    7,    6,    5,    4,    3,    2,
          1,   -1,   -2,   -3,   -4,   -5,   -6,   -7,   -8,   -9,  -10,
        -11,  -12,  -13,  -14,  -15,  -16,  -17,  -18,  -19,  -20,  -21,
        -22,  -23,  -24,  -25,  -26,  -27,  -28,  -29,  -30,  -31,  -32,
        -33,  -34,  -35,  -36,  -37,  -38,  -39,  -40,  -41,  -42,  -43,
        -44,  -45,  -46,  -47,  -48,  -49,  -50,  -51,  -52,  -53,  -54,
        -55,  -56,  -57,  -58,  -59,  -60,  -61,  -62,  -63,  -64,  -65,
        -66,  -67,  -68,  -69,  -70,  -71,   67,   68,   69,   70,  -72,
         71,   72,   73,   74,   75,  -73,   76,   

In [ ]:
df2=df.groupby(['Longitude','Latitude']).CM.apply(countzero)

In [39]:
df2

Longitude  Latitude
0          0           1.000000
11         16          0.039409
           17          0.001684
           18          0.052419
           19          0.321212
           20          0.650000
12         16          0.010260
           17          0.000000
           18          0.164643
           19          0.586475
           20          0.562278
           21          1.000000
           22          1.000000
           23          0.998369
           24          0.684964
           25          0.816068
           26          0.382114
13         15          0.000000
           16          0.000000
           17          0.001453
           18          0.174857
           19          0.318465
           20          0.180676
           21          0.811667
           22          1.000000
           23          0.995710
           24          0.618995
           25          0.328233
           26          0.741529
           27          0.999555
                    

In [40]:
df3=df2.reset_index()
df3

,Longitude,Latitude,CM
0,0,0,1.000000
1,11,16,0.039409
2,11,17,0.001684
3,11,18,0.052419
4,11,19,0.321212
5,11,20,0.650000
6,12,16,0.010260
7,12,17,0.000000
8,12,18,0.164643
9,12,19,0.586475


In [41]:
import pandas as pd
combs=[]
for x in range(-89,91):
    for y in range(-179,181):
        combs.append((x, y))


In [42]:
df_1=pd.DataFrame(combs)
df_1.columns=['Latitude','Longitude']


In [44]:
df4=pd.merge(df_1, df3,on=('Longitude','Latitude'), how='left')
df4

,Latitude,Longitude,CM
0,-89,-179,NaN
1,-89,-178,NaN
2,-89,-177,NaN
3,-89,-176,NaN
4,-89,-175,NaN
5,-89,-174,NaN
6,-89,-173,NaN
7,-89,-172,NaN
8,-89,-171,NaN
9,-89,-170,NaN


In [45]:
df5=df4['CM'].values

b=df5.reshape(180,360)


In [46]:
total_cloud_fraction = b
out_name = '/Users/dprakas1/Desktop/reshape.hdf5'
def save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd):
    f=h5py.File(out_name,'w')
    PCentry=f.create_dataset('CF',data=total_cloud_fraction)
    PCentry.dims[0].label='lat_bnd'
    #PCentry.dims[1].label='lon_bnd'

    PC=f.create_dataset('lat_bnd',data=lat_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Latitude_boundaries'

    PC=f.create_dataset('lon_bnd',data=lon_bnd)
    PC.attrs['units']='degrees'
    PC.attrs['long_name']='Longitude_boundaries'
    f.close()
    print(out_name+' Saved!!')

save_hdf(out_name,total_cloud_fraction,lat_bnd,lon_bnd)



/Users/dprakas1/Desktop/reshape.hdf5 Saved!!


In [48]:
#for one day's data
import h5py
import numpy as np
from comparisons import readData, doPlot
benchmark_p="/Users/dprakas1/Documents/"
CF_BMK,_,_=readData(benchmark_p+"MODAgg_3var_parMonth_20080101.hdf5")#Benchmark

f=h5py.File('/Users/dprakas1/Desktop/reshape.hdf5','r')
CF_res=f['CF'][:]
fig1,fig1_ttl=doPlot(CF_res,CF_BMK,'Comparison')



OSError: Unable to open file (unable to open file: name = '/Users/dprakas1/Documents/MODAgg_3var_parMonth_20080101.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)